In [7]:
import geopandas

# Replace 'path_to_your_file' with the actual path to your SAIS_areaPolygons file
file_path = '../data/processed/SAIS_areaPolygons.geojson'

# Read the data
gdf = geopandas.read_file(file_path)

# Display the first few rows of the data
gdf.head()


,name,geometry
0,Creag Meagaidh,"POLYGON ((-4.41507 56.9744, -4.44116 56.96485,..."
1,Glencoe,"POLYGON ((-5.25865 56.65222, -5.24578 56.67402..."
2,Lochaber,"POLYGON ((-5.08972 56.83065, -5.06397 56.83384..."
3,Northern Cairngorms,"POLYGON ((-3.78343 57.20349, -3.75545 57.20795..."
4,Southern Cairngorms,"POLYGON ((-3.37868 57.01625, -3.35379 57.0069,..."
